# --- Day 7: Camel Cards ---
## Part 1
Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

    Five of a kind, where all five cards have the same label: AAAAA
    Four of a kind, where four cards have the same label and one card has a different label: AA8AA
    Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
    Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
    Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
    One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
    High card, where all cards' labels are distinct: 23456

Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483

This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

    32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
    KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
    T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?


In [96]:
from collections import OrderedDict

test_data = {
    "32T3K": 765,
    "T55J5": 684,
    "KK677": 28,
    "KTJJT": 220,
    "QQQJA": 483
}
real_data = {i.split(' ')[0]: int(i.split(' ')[1]) for i in open("input.txt").read().splitlines()}

data = real_data

cards = ["A", "K", "Q", "J", "T", "9",
         "8", "7", "6", "5", "4", "3", "2"]
card_value = {card: len(cards)-i for i,card in enumerate(cards)}

def get_hand_points(hand: str) -> int:
    # Points will be 7 to 1 from strongest to weakest type
    counts = {card: 0 for card in cards}
    for c in hand:
        counts[c] +=1
    if any(p == 5 for p in counts.values()): return 7
    if any(p == 4 for p in counts.values()): return 6
    if any(p == 3 for p in counts.values()) and\
       any(p == 2 for p in counts.values()) : return 5
    if any(p == 3 for p in counts.values()): return 4
    if sum(p == 2 for p in counts.values()) == 4: return 3
    if any(p == 2 for p in counts.values()): return 2
    return 1

def get_max_char(chars: list) -> str:
    max_char = chars[0]
    for c in chars[1:]:
        max_char = c if card_value[c] > card_value[max_char] else max_char
    return max_char

def compare_equal_points_hands(hands: list) -> list:
    for i in range(len(hands[0])):
        chars = [h[i] for h in hands]
        mc = get_max_char(chars)
        if chars.count(mc)==1:
            return next(h for h in hands if h[i] == mc)
        else:
            hands = [h for h in hands if h[i] == mc]
    # We should never arrive to this return
    return 0

result = {hand: get_hand_points(hand) for hand in data}
result = OrderedDict(sorted(result.items(),
                            key=lambda x: x[1], reverse=True))

prev_value=next(iter(result.values()))
compare = []
rank = {}
n = len(result)

for hand, value in result.items():
    if value == prev_value:
        compare.append(hand)
    else:
        while len(compare) > 1:
            biggest = compare_equal_points_hands(compare)
            rank[biggest] = n
            n -= 1
            compare.remove(biggest)
        rank[compare[0]] = n
        n -= 1
        compare = [hand]
    prev_value = value
    
while len(compare) > 1:
    biggest = compare_equal_points_hands(compare)
    rank[biggest] = n
    n -= 1
    compare.remove(biggest)
rank[compare[0]] = n

print(len(rank),len(data))

winnings = sum(rank[hand]*bid for hand,bid in data.items())

print(rank)
#print(data)
print("The total winnings are: ", winnings)


1000 1000
{'JJJJJ': 1000, 'AAA7A': 999, 'AA8AA': 998, 'AA4AA': 997, 'AA3AA': 996, 'AJAAA': 995, 'A3333': 994, 'KAAAA': 993, 'KKAKK': 992, 'KKKK8': 991, 'KKQKK': 990, 'KKTKK': 989, 'KK9KK': 988, 'KJKKK': 987, 'K9999': 986, 'K8888': 985, 'K7KKK': 984, 'K6KKK': 983, 'QQQKQ': 982, 'QQQQ5': 981, 'QQJQQ': 980, 'Q9QQQ': 979, 'Q6QQQ': 978, 'Q6666': 977, 'Q4QQQ': 976, 'JJJJ8': 975, 'J9999': 974, 'J7777': 973, 'TKTTT': 972, 'TQQQQ': 971, 'TTATT': 970, 'TTTTJ': 969, 'TTTT3': 968, 'TT8TT': 967, 'TT4TT': 966, 'T2222': 965, '999Q9': 964, '9999T': 963, '99699': 962, '99299': 961, '98999': 960, '95999': 959, '8A888': 958, '8Q888': 957, '888J8': 956, '8888T': 955, '88887': 954, '88488': 953, '82222': 952, '77K77': 951, '77977': 950, '777T7': 949, '7777A': 948, '7777Q': 947, '77577': 946, '72777': 945, '6TTTT': 944, '6T666': 943, '66866': 942, '6666J': 941, '66667': 940, '66663': 939, '66466': 938, '63333': 937, '62666': 936, '5AAAA': 935, '5K555': 934, '55655': 933, '555Q5': 932, '55575': 931, '5555J':